# Cleaning the data

In this notebook, I compile all the text from all the books into one dataframe, then do some cleaning and add some columns.

### Import things

In [1]:
import os
import string
import pandas as pd

In [2]:
try:
    os.mkdir('./CSVs')
except:
    pass

In [3]:
#look at the names of some of the files in the txt folder
txt_path = './TXT'
books = os.listdir(txt_path)
books[0:5]

['450 From Paddington - Agatha Christie.txt',
 'A Caribbean Mystery_ A Miss Mar - Agatha Christie.txt',
 'A Murder Is Announced_ A Miss M - Agatha Christie.txt',
 'A Pocket Full of Rye - Agatha Christie.txt',
 'After the Funeral_ A Hercule Po - Agatha Christie.txt']

### Concatenating all the text from all the books:

In [4]:
books_list = []
for file in books:  
    d={}
    book = os.path.join('./TXT', file)
    text = pd.read_csv(book, sep='\n', header=None)[0].str.cat()
    d['title'] = file[:-22].split('_')[0]
    d['text'] = text
    books_list.append(d)
df = pd.DataFrame(books_list)
df.head()

,title,text
0,450 From Paddington,Agatha Christie4:50 from PaddingtonA Miss Marp...
1,A Caribbean Mystery,Agatha ChristieA Caribbean MysteryA Miss Marpl...
2,A Murder Is Announced,Agatha ChristieA Murder Is AnnouncedA Miss Mar...
3,A Pocket Full of Rye,A Pocket Full of RyeA Miss Marple MysteryAgath...
4,After the Funeral,Agatha ChristieAfter the FuneralA Hercule Poir...


In [5]:
#save the df for later in case things go awry
df.to_csv('./CSVs/books.csv', index=False)

In [6]:
#reload the df
df = pd.read_csv('./CSVs/books.csv')

### Extracting the copyright dates for the book:

In [7]:
copyright_list = []
for i in range(len(df['text'])):
    for word in range(len(df['text'][i])):
        d = {}
        if df['text'][i][word] == '©':
            d['title'] = df['title'][i] 
            d['copyright'] = df['text'][i][word: word + 14]
            copyright_list.append(d)

In [8]:
copyright_df = pd.DataFrame(copyright_list)
copyright_df.head(9)

,title,copyright
0,450 From Paddington,© 2011 Agatha
1,450 From Paddington,© 1957. Publis
2,450 From Paddington,© MAY 2011 ISB
3,A Caribbean Mystery,© 2011 Agatha
4,A Caribbean Mystery,© 1965. Publis
5,A Caribbean Mystery,© MAY 2011 ISB
6,A Murder Is Announced,© 2011 Agatha
7,A Murder Is Announced,© 1950. Publis
8,A Murder Is Announced,© MAY 2011 ISB


#### It looks like each book has 3 copy right dates. I only want the original copyright date
I assume the original copyright date is the earliest year

In [9]:
copyright_df['year'] = copyright_df['copyright'].map(lambda x: ''.join([y for y in list(x) if y in '0123456789']))

In [10]:
#drop blank years
copyright_df = copyright_df[copyright_df['year'] != '']

In [11]:
#convert year strings into integers
copyright_df['year'] = copyright_df['year'].astype(int)

In [12]:
#Her last book was published in 1976 so I'll filter out all years after that
#Also I'll reset the index here
copyright_df = copyright_df[copyright_df['year'] <= 1976].reset_index(drop=True)

In [13]:
#also get rid of any years that are like, 201:
copyright_df = copyright_df[copyright_df['year'] > 1000]

In [14]:
#sort by name, then sort by year
copyright_df = copyright_df.sort_values(['title', 'year'], ascending = [True, True])
copyright_df.head(20)

,title,copyright,year
0,450 From Paddington,© 1957. Publis,1957
1,A Caribbean Mystery,© 1965. Publis,1965
2,A Murder Is Announced,© 1950. Publis,1950
3,A Pocket Full of Rye,© 1954. Publis,1954
4,After the Funeral,© 1953 Agatha,1953
5,After the Funeral,© 1953. Publis,1953
6,Appointment With Death,© 1938 Agatha,1938
7,At Bertram's Hotel,© 1966. Publis,1966
8,By the Pricking of My Thumbs,© 1968 Agatha,1968
9,By the Pricking of My Thumbs,© 1968. Publis,1968


In [15]:
copyright_df = copyright_df.drop_duplicates(subset='title', keep='first').reset_index()
len(copyright_df)

82

In [16]:
len(df)

94

In [17]:
df = pd.merge(df, copyright_df, left_on='title', right_on = 'title', how='outer')

In [18]:
df = df.drop(['copyright','index'], axis=1)

### Look through the books that had been dropped
Research what these books are/when they were published and decide whether or not to keep them

In [19]:
#looks like I accidentally deleted some books while looking for the right year
df[df['year'].isnull()]

,title,text,year
5,An Autobiography,Agatha ChristieAn AutobiographyContentsCoverPr...,NaN
6,And Then There Were None,Agatha ChristieAnd Then There Were NoneDedicat...,NaN
9,Black Coffee,Agatha ChristieBlack Coffeeadapted as a novel ...,NaN
27,Hercule Poirot and the Greensho,ContentsForewordHercule Poirot and the Greensh...,NaN
28,Hercule Poirot's Casebook,HERCULE POIROT'S CASEBOOKAgatha Christie is kn...,NaN
33,Miss Marple's Final Cases,Miss Marple’s Final CasesContentsAbout Agatha ...,NaN
34,Miss Marple,ContentsAuthor’s ForewordFrom The Thirteen Pro...,NaN
55,Spider's Web,Agatha ChristieSpider’s WebNovelised by Charle...,NaN
63,The Harlequin Tea Set and Other,The HarlequinTea Set andOther StoriesContentsC...,NaN
69,The Mirror Crack'd From Side to,Agatha ChristieThe Mirror Crack’d from Side to...,NaN


In [20]:
missing = df[df['year'].isnull()]['title'].tolist()

### Remake the copyright df ):

In [21]:
c_df = pd.DataFrame(copyright_list)
c_df['missing'] = c_df['title'].map(lambda x: x if x in missing else "")
c_df = c_df[c_df['missing'] != '']

In [22]:
c_df['year'] = c_df['copyright'].map(lambda x: ''.join([y for y in list(x) if y in '0123456789']))

In [23]:
#this is the dataframe with the missing books
c_df = c_df.sort_values(['title', 'year'], ascending = [True, True])
c_df

,title,copyright,missing,year
15,An Autobiography,© 1977 Agatha,An Autobiography,1977
16,An Autobiography,© March 2010 I,An Autobiography,2010
17,And Then There Were None,© 2009 Agatha,And Then There Were None,2009
18,And Then There Were None,© 2009. Publis,And Then There Were None,2009
19,And Then There Were None,© MAY 2011 ISB,And Then There Were None,2011
26,Black Coffee,"© 1982, 1999 b",Black Coffee,19821999
25,Black Coffee,© 1997 Agatha,Black Coffee,1997
27,Black Coffee,© 2010 ISBN: 9,Black Coffee,20109
78,Hercule Poirot and the Greensho,© Mathew Prich,Hercule Poirot and the Greensho,
79,Hercule Poirot and the Greensho,© John Curran,Hercule Poirot and the Greensho,


In [24]:
df['title'].tolist()

['450 From Paddington',
 'A Caribbean Mystery',
 'A Murder Is Announced',
 'A Pocket Full of Rye',
 'After the Funeral',
 'An Autobiography',
 'And Then There Were None',
 'Appointment With Death',
 "At Bertram's Hotel",
 'Black Coffee',
 'By the Pricking of My Thumbs',
 'Cards on the Table',
 'Cat Among the Pigeons',
 'Crooked House',
 'Curtain',
 "Dead Man's Folly",
 'Death Comes as the End',
 'Death in the Clouds',
 'Death on the Nile',
 'Destination Unknown',
 'Double Sin and Other Stories',
 'Dumb Witness',
 'Elephants Can Remember',
 'Endless Night',
 'Evil Under the Sun',
 'Five Little Pigs',
 "Hallowe'en Party",
 'Hercule Poirot and the Greensho',
 "Hercule Poirot's Casebook",
 "Hercule Poirot's Christmas",
 'Hercule Poirot',
 'Hickory Dickory Dock',
 'Lord Edgware Dies',
 "Miss Marple's Final Cases",
 'Miss Marple',
 "Mrs. McGinty's Dead",
 'Murder in Mesopotamia',
 'Murder in the Mews',
 'Murder Is Easy',
 'Murder on the Orient Express',
 'N or M',
 'Nemesis',
 'One, Two, Buc

In [25]:
#change And Then There Were None to 1939
df.loc[6, 'year'] = 1939
df[df['title'] == 'And Then There Were None']

,title,text,year
6,And Then There Were None,Agatha ChristieAnd Then There Were NoneDedicat...,1939.0


### I'm going to drop the autobiography and do a seperate analysis by itself because this book is different:

The autobiography was written over the course of 15 years, and she didn't really edit it. However, the publishers admitted to cleaning up the text.

source: https://en.wikipedia.org/wiki/Agatha_Christie:_An_Autobiography


In [26]:
df = df.drop(5)

In [27]:
#drop black coffee because it's adapted as a novel by Charles Osborne
df = df.drop(9)

In [28]:
#source: https://www.agathachristie.com/stories/hercule-poirot-and-the-greenshore-folly
df.loc[27, 'year'] = 1954
df[df['title'] == 'Hercule Poirot and the Greensho']

,title,text,year
27,Hercule Poirot and the Greensho,ContentsForewordHercule Poirot and the Greensh...,1954.0


In [29]:
#drop Hercule Poirot's Casebook because it's a collection of other stories already included in this list:
df = df.drop(28)

In [30]:
df.loc[69, 'year'] = 1962
df[df['title'] == "The Mirror Crack'd From Side to"]

,title,text,year
69,The Mirror Crack'd From Side to,Agatha ChristieThe Mirror Crack’d from Side to...,1962.0


In [31]:
#drop Miss Marple, Miss Marple's final cases because they're collections and they don't give the years when they were published
#drop Spider's Web because it's a play
#drop the unexpected guest because it's an adaptation by Charles Osborne
#drop while the light lasts, because it's a collection of other stories already in here

df = df.drop([33,34, 55, 84, 92])

In [32]:
#drop from main one and I'll deal with it later, because they're also collections but they give the dates.
df =df.drop([30,63])

In [33]:
# Drop "Poirot's Early Cases", because it's a collection of stories:
df =df.drop(49)

In [34]:
#save
df.to_csv('./CSVs/books_df_cleaned.csv', index=False)

### Open up harlequin short stories and seperate out all the stories:
Thankfully the short stories are all seperated by their names in all caps, so I can just split by the all caps names

In [35]:
harlequin = pd.read_csv('./TXT/The Harlequin Tea Set and Other - Agatha Christie.txt', sep='\n', header=None)[0].str.cat()

In [36]:
#list of short stories in the harlequin tea set
short_stories = ['The Edge', 
                 'The Actress', 
                 'While the Light Lasts', 
                 'The House of Dreams', 
                 'The Lonely God',
                 'Manx Gold',
                 'Within a Wall', 
                 'The Mystery of the Spanish Chest', 
                 'The Harlequin Tea Set']

#make them all caps
short_stories = [x.upper() for x in short_stories]

In [37]:
short_stories

['THE EDGE',
 'THE ACTRESS',
 'WHILE THE LIGHT LASTS',
 'THE HOUSE OF DREAMS',
 'THE LONELY GOD',
 'MANX GOLD',
 'WITHIN A WALL',
 'THE MYSTERY OF THE SPANISH CHEST',
 'THE HARLEQUIN TEA SET']

In [38]:
stories = []
for story in range(len(short_stories)-1):
    s = {}
    s['title'] = short_stories[story]
    s['text'] = harlequin.split(short_stories[story])[1].split(short_stories[story+1])[0]
    stories.append(s)

In [39]:
last = harlequin.split('THE HARLEQUIN TEA SET')[1].split('About the Author')[0]

s = {}
s['title'] = 'THE HARLEQUIN TEA SET'
s['text'] = last

stories.append(s)

In [40]:
stories_df = pd.DataFrame(stories)
stories_df

,title,text
0,THE EDGE,“The Edge” was first published in Pearson’s Ma...
1,THE ACTRESS,“The Actress” was first published as “A Trap f...
2,WHILE THE LIGHT LASTS,“While the Light Lasts” was first published in...
3,THE HOUSE OF DREAMS,“The House of Dreams” was first published in T...
4,THE LONELY GOD,“The Lonely God” was first published in Royal ...
5,MANX GOLD,“Manx Gold” was first published in The Daily D...
6,WITHIN A WALL,“Within a Wall” was first published in Royal M...
7,THE MYSTERY OF THE SPANISH CHEST,“The Mystery of the Spanish Chest” is an expan...
8,THE HARLEQUIN TEA SET,“The Harlequin Tea Set” was first published in...


In [41]:
#split out first sentence, then only save the numbers
stories_df['year'] = stories_df['text'].map(lambda x: x.split('.')[0]).map(lambda x: ''.join([y for y in list(x) if y in '0123456789']))

In [42]:
stories_df

,title,text,year
0,THE EDGE,“The Edge” was first published in Pearson’s Ma...,1927
1,THE ACTRESS,“The Actress” was first published as “A Trap f...,1923
2,WHILE THE LIGHT LASTS,“While the Light Lasts” was first published in...,1924
3,THE HOUSE OF DREAMS,“The House of Dreams” was first published in T...,1926
4,THE LONELY GOD,“The Lonely God” was first published in Royal ...,1926
5,MANX GOLD,“Manx Gold” was first published in The Daily D...,23281930
6,WITHIN A WALL,“Within a Wall” was first published in Royal M...,1925
7,THE MYSTERY OF THE SPANISH CHEST,“The Mystery of the Spanish Chest” is an expan...,1932
8,THE HARLEQUIN TEA SET,“The Harlequin Tea Set” was first published in...,1971


In [43]:
#fix manx gold:
stories_df.loc[5, 'year'] = 1930
stories_df

,title,text,year
0,THE EDGE,“The Edge” was first published in Pearson’s Ma...,1927
1,THE ACTRESS,“The Actress” was first published as “A Trap f...,1923
2,WHILE THE LIGHT LASTS,“While the Light Lasts” was first published in...,1924
3,THE HOUSE OF DREAMS,“The House of Dreams” was first published in T...,1926
4,THE LONELY GOD,“The Lonely God” was first published in Royal ...,1926
5,MANX GOLD,“Manx Gold” was first published in The Daily D...,1930
6,WITHIN A WALL,“Within a Wall” was first published in Royal M...,1925
7,THE MYSTERY OF THE SPANISH CHEST,“The Mystery of the Spanish Chest” is an expan...,1932
8,THE HARLEQUIN TEA SET,“The Harlequin Tea Set” was first published in...,1971


In [44]:
#get rid of the first sentence of the text:
stories_df['text'] = stories_df['text'].map(lambda x: '.'.join(x.split('.')[1:]))

In [45]:
#the last story had 2 sentences:
stories_df.loc[8, 'text'] = '.'.join((stories_df['text'][8].split('.')[1:]))

In [46]:
stories_df.to_csv('./CSVs/short_stories_df.csv', index=False)

### Ok now split up all the Hercule Poirot short stories

In [47]:
hercule = pd.read_csv('./TXT/Hercule Poirot_ The Complete Sh - Agatha Christie.txt', sep='\n', header=None)[0].str.cat()

In [48]:
hercule_stories = ['The Affair at the Victory Ball',
                   'The Jewel Robbery at the Grand Metropolitan',
                   'The King of Clubs',
                   'The Disappearance of Mr. Davenheim',
                   'The Plymouth Express',
                   'The Adventure of “The Western Star”',
                   'The Tragedy at Marsdon Manor',
                   'The Kidnapped Prime Minister',
                   'The Million Dollar Bond Robbery',
                   'The Adventure of the Cheap Flat',
                   'The Mystery of Hunter’s Lodge',
                   'The Chocolate Box',
                   'The Adventure of the Egyptian Tomb',
                   'The Veiled Lady',
                   'The Adventure of Johnnie Waverly',
                   'The Market Basing Mystery',
                   'The Adventure of the Italian Nobleman',
                   'The Case of the Missing Will',
                   'The Incredible Theft',
                   'The Adventure of the Clapham Cook',
                   'The Lost Mine',
                   'The Cornish Mystery',
                   'The Double Clue',
                   'The Theft of the Royal Ruby',
                   'The Lemesurier Inheritance',
                   'The Under Dog',
                   'Double Sin',
                   'Wasps’ Nest',
                   'The Third Floor Flat',
                   'The Mystery of the Baghdad Chest',
                   'Dead Man’s Mirror',
                   'How Does Your Garden Grow?',
                   'Problem at Sea',
                   'Triangle at Rhodes',
                   'Murder in the Mews',
                   'Yellow Iris',
                   'The Dream',
                   'The Labors of Hercules',
                   'The Nemean Lion',
                   'The Lernean Hydra',
                   'The Arcadian Deer',
                   'The Erymanthian Boar',
                   'The Augean Stables',
                   'The Stymphalean Birds',
                   'The Cretan Bull',
                   'The Horses of Diomedes',
                   'The Girdle of Hyppolita',
                   'The Flock of Geryon',
                   'The Apples of the Hesperides',
                   'The Capture of Cerberus', 
                   'Four and Twenty Blackbirds']

In [49]:
hercule_stories = [x.upper() for x in hercule_stories]

In [50]:
h_stories = []
for title in range(len(hercule_stories)-1):
    s = {}
    s['title'] = hercule_stories[title]
    s['text'] = hercule.split(hercule_stories[title])[1].split(hercule_stories[title+1])[0]
    h_stories.append(s)
    

In [51]:
last = hercule.split('FOUR AND TWENTY BLACKBIRDS')[1].split('About the Author')[0]

s = {}
s['title'] = 'FOUR AND TWENTY BLACKBIRDS'
s['text'] = last

h_stories.append(s)

In [52]:
hercule_df = pd.DataFrame(h_stories)
hercule_df.head()

,title,text
0,THE AFFAIR AT THE VICTORY BALL,“The Affair at the Victory Ball” was first pub...
1,THE JEWEL ROBBERY AT THE GRAND METROPOLITAN,“The Jewel Robbery at the Grand Metropolitan” ...
2,THE KING OF CLUBS,“The King of Clubs” was first published as “Th...
3,THE DISAPPEARANCE OF MR. DAVENHEIM,“The Disappearance of Mr. Davenheim” was first...
4,THE PLYMOUTH EXPRESS,“The Plymouth Express” was first published as ...


In [53]:
hercule_df['year'] = hercule_df['text'].map(lambda x: x.split('.')[0]).map(lambda x: ''.join([y for y in list(x) if y in '0123456789'])[-4:])

In [54]:
hercule_df.loc[3, 'year'] = 1923
hercule_df.head()

,title,text,year
0,THE AFFAIR AT THE VICTORY BALL,“The Affair at the Victory Ball” was first pub...,1923
1,THE JEWEL ROBBERY AT THE GRAND METROPOLITAN,“The Jewel Robbery at the Grand Metropolitan” ...,1923
2,THE KING OF CLUBS,“The King of Clubs” was first published as “Th...,1923
3,THE DISAPPEARANCE OF MR. DAVENHEIM,“The Disappearance of Mr. Davenheim” was first...,1923
4,THE PLYMOUTH EXPRESS,“The Plymouth Express” was first published as ...,1923


In [55]:
stories_df.to_csv('./CSVs/hercule_stories.csv', index=False)

### Cleaning the df with the books

In [56]:
#split by 'about the publisher'?
df['text'] = df['text'].map(lambda x: ' '.join(x.split('About the Publisher')[1:]))

In [57]:
df = df.reset_index(drop=True)

In [58]:
#getting rid of all the stuff at the end of the book
df['text'] = df['text'].map(lambda x: x.lower().split('about the author')[0])
df['text'] = df['text'].map(lambda x: x.split('* * *')[0])
df['text'] = df['text'].map(lambda x: x.split('e-book')[0])

In [59]:
#it's mostly clean now, just with the first word mushed together.
df.head()

,title,text,year
0,450 From Paddington,onemrs. mcgillicuddy panted along the platform...,1957.0
1,A Caribbean Mystery,onemajor palgrave tells a story“take all this ...,1965.0
2,A Murder Is Announced,onea murder is announcedibetween 7:30 and 8:30...,1950.0
3,A Pocket Full of Rye,chapter oneit was miss somers’s turn to make t...,1954.0
4,After the Funeral,oneiold lanscombe moved totteringly from room ...,1953.0


### Combine short stories dfs with the books df:

In [60]:
new_df = pd.concat([df, stories_df, hercule_df],axis=0)

In [61]:
new_df['title'] = new_df['title'].map(lambda x: x.lower())
new_df['text'] = new_df['text'].map(lambda x: x.lower())

In [62]:
new_df.head()

,title,text,year
0,450 from paddington,onemrs. mcgillicuddy panted along the platform...,1957
1,a caribbean mystery,onemajor palgrave tells a story“take all this ...,1965
2,a murder is announced,onea murder is announcedibetween 7:30 and 8:30...,1950
3,a pocket full of rye,chapter oneit was miss somers’s turn to make t...,1954
4,after the funeral,oneiold lanscombe moved totteringly from room ...,1953


In [63]:
new_df = new_df[new_df['text'] != '']

### More cleaning:
Going to perform the cleaning that they did in the paper:
> After all punctuation, apostrophes, and hyphens were deleted, each text was divided into 10,000-word segments.

In [64]:
new_df['text'] = new_df['text'].map(lambda x: x.lower())

In [65]:
#save period because I want a rough sentence count
new_df['text'] = new_df['text'].map(lambda x: ''.join([y for y in list(x.lower()) if y in string.ascii_lowercase + ' .']))

In [66]:
new_df['sentence_count'] = new_df['text'].map(lambda x: len(x.split('.')))

In [67]:
#now we can get rid of periods
new_df['text'] = new_df['text'].map(lambda x: ''.join([y for y in list(x.lower()) if y in string.ascii_lowercase + ' ']))

In [68]:
new_df = new_df.reset_index(drop=True)

### OK so it turns out Sleeping Murder and Curtain were written in 1940, but published after she passed away. 
I need to fix the date in the dataframe  
source: https://en.wikipedia.org/wiki/Sleeping_Murder
source: https://en.wikipedia.org/wiki/Curtain_(novel)

In [71]:
new_df.loc[12]

title                                                       curtain
text              oneiwho is there who has not felt a sudden sta...
year                                                           1975
sentence_count                                                 5126
Name: 12, dtype: object

In [72]:
new_df.loc[43]

title                                               sleeping murder
text              onea housegwenda reed stood shivering a little...
year                                                           1976
sentence_count                                                 5356
Name: 43, dtype: object

In [73]:
new_df.loc[12, 'year'] = 1940
new_df.loc[43, 'year'] = 1940

### I think it's mostly clean now!

In [74]:
new_df['year'] = new_df['year'].astype(int)

In [75]:
new_df.to_csv('./CSVs/all_clean.csv', index=False)